In [ ]:
!pip install scapy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 29.8 MB/s eta 0:00:00


In [ ]:
### Breeze-v0.0.1 Test

#===Imports
from scapy.all import rdpcap, IP, TCP, UDP, DNS, DNSQR
import pandas as pd
from collections import Counter
#===

#===Globals
path = './ping_dns_test.pcapng'
#===

## Testing PCAP access
print(f'[-] Accessing PCAP at path: {"{"}{path}{"}"}`... ', end='')
try:
  packets = rdpcap(path)
  print(f'DONE.')
except Exception as e:
  print(f'\n\t[!] Could not access PCAP at path: `{path}`. Aborting...')
  exit(0)

### Getting information about PCAP
# rows, start/end time, date

[-] Accessing PCAP at path: {./ping_dns_test.pcapng}`... DONE.


In [ ]:
for pkt in packets:
  print(pkt)

Ether / IP / TCP 20.69.136.49:443 > 66.71.101.209:54089 FA
Ether / IP / TCP 66.71.101.209:54089 > 20.69.136.49:443 A
Ether / IP / TCP 162.159.130.234:443 > 66.71.101.209:53941 PA / Raw
Ether / IP / TCP 162.159.130.234:443 > 66.71.101.209:53941 PA / Raw
Ether / IP / TCP 66.71.101.209:53941 > 162.159.130.234:443 A
Ether / IP / TCP 162.159.130.234:443 > 66.71.101.209:53941 PA / Raw
Ether / IP / TCP 66.71.101.209:53941 > 162.159.130.234:443 A
Ether / IP / TCP 162.159.130.234:443 > 66.71.101.209:53941 PA / Raw
Ether / IP / TCP 66.71.101.209:53941 > 162.159.130.234:443 A
Ether / IP / TCP 162.159.130.234:443 > 66.71.101.209:53941 PA / Raw
Ether / IP / TCP 66.71.101.209:53941 > 162.159.130.234:443 A
Ether / IP / TCP 162.159.130.234:443 > 66.71.101.209:53941 PA / Raw
Ether / IP / TCP 66.71.101.209:53941 > 162.159.130.234:443 A
Ether / IP / TCP 162.159.130.234:443 > 66.71.101.209:53941 PA / Raw
Ether / IP / TCP 66.71.101.209:53941 > 162.159.130.234:443 A
Ether / IP / TCP 66.71.101.209:53958 > 17

In [ ]:
from datetime import datetime

timestamp = packets[0].time
converted_timestamp = datetime.fromtimestamp(timestamp)
print(f'original : {timestamp}')
print(f'converted: {converted_timestamp}')

TypeError: 'EDecimal' object cannot be interpreted as an integer

In [ ]:
from datetime import datetime

datetime.fromtimestamp(1770132796.3301687)

datetime.datetime(2026, 2, 3, 15, 33, 16, 330169)

# working time stamp comparison
but without seconds conversion. need to continue with UTC date timestamp.

In [ ]:
from datetime import datetime

p0 = packets[0]
pLAST = packets[len(packets) - 1]
p0_t = p0.time
pLAST_t = pLAST.time

delta = pLAST_t - p0_t
print(f'delta p0, pLAST: {float(delta):.6f} seconds')

delta p0, pLAST: 77.720585 seconds


In [ ]:
# sample

from scapy.all import rdpcap, IP, TCP, UDP, DNS, DNSQR
import pandas as pd
from collections import Counter

PCAP_FILE = "cap.pcapng"

# ----------------------------
# Load PCAP
# ----------------------------
packets = rdpcap(PCAP_FILE)

rows = []
iocs = {
    "ips": set(),
    "domains": set(),
    "ports": set()
}

# ----------------------------
# Packet parsing
# ----------------------------
for pkt in packets:
    if IP in pkt:
        src_ip = pkt[IP].src
        dst_ip = pkt[IP].dst
        proto = pkt[IP].proto

        src_port = None
        dst_port = None
        dns_query = None

        # TCP
        if TCP in pkt:
            src_port = pkt[TCP].sport
            dst_port = pkt[TCP].dport
            iocs["ports"].add(dst_port)

        # UDP
        elif UDP in pkt:
            src_port = pkt[UDP].sport
            dst_port = pkt[UDP].dport
            iocs["ports"].add(dst_port)

        # DNS
        if pkt.haslayer(DNS) and pkt.haslayer(DNSQR):
            dns_query = pkt[DNSQR].qname.decode(errors="ignore")
            iocs["domains"].add(dns_query)

        # Track IP IOCs
        iocs["ips"].update([src_ip, dst_ip])

        rows.append({
            "src_ip": src_ip,
            "dst_ip": dst_ip,
            "src_port": src_port,
            "dst_port": dst_port,
            "protocol": proto,
            "dns_query": dns_query,
            "packet_len": len(pkt)
        })

for pkt in packets:
  print(pkt)

# ----------------------------
# Build DataFrame
# ----------------------------
df = pd.DataFrame(rows)

# ----------------------------
# Feature analysis
# ----------------------------
top_src_ips = Counter(df["src_ip"]).most_common(10)
top_dst_ips = Counter(df["dst_ip"]).most_common(10)
top_ports = Counter(df["dst_port"].dropna()).most_common(10)
top_domains = Counter(df["dns_query"].dropna()).most_common(10)

# ----------------------------
# Output summary
# ----------------------------
print("\n=== IOC SUMMARY ===")
print(f"Unique IPs: {len(iocs['ips'])}")
print(f"Unique Domains: {len(iocs['domains'])}")
print(f"Unique Ports: {len(iocs['ports'])}")

print("\nTop Source IPs:")
for ip, count in top_src_ips:
    print(f"{ip}: {count}")

print("\nTop Destination IPs:")
for ip, count in top_dst_ips:
    print(f"{ip}: {count}")

print("\nTop Destination Ports:")
for port, count in top_ports:
    print(f"{port}: {count}")

print("\nTop Queried Domains:")
for domain, count in top_domains:
    print(f"{domain}: {count}")

# ----------------------------
# Save for analysis / ML
# ----------------------------
df.to_csv("flow_packet_features.csv", index=False)
```

---

In [ ]:
!pip install scapy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 28.5 MB/s eta 0:00:00


In [ ]:
### Breeze-v0.0.1 Test

#===Imports
from scapy.all import rdpcap, IP, TCP, UDP, DNS, DNSQR
import pandas as pd
from datetime import datetime

from collections import Counter
#===

#===Globals
path = './ping_dns_test.pcapng'
#===

## Testing PCAP access
print(f'[-] Accessing PCAP at path: {"{"}{path}{"}"}`... ', end='')
try:
  packets = rdpcap(path)
  print(f"typeof packets: {type(packets)}")
  print(f'DONE.')
except Exception as e:
  print(f'\n\t[!] Could not access PCAP at path: `{path}`. Aborting...')
  exit(0)

### Getting information about PCAP
# rows, start/end time, date

number_of_packets = len(packets)
first_packet = packets[0]
last_packet = packets[number_of_packets - 1]

print(datetime.fromtimestamp(1770132796.3301687))

p0 = packets[0]
print(f'type of p0: {type(p0)}')
pLAST = packets[len(packets) - 1]
p0_t = p0.time
pLAST_t = pLAST.time

print(f'timestamp first_packet: {first_packet.time}')
print(f'timestamp last_packet: {last_packet.time}')
# float() for EDecimal conversion. EDecimal is for delta time, float is for datetime.
print(f'UTC converted time first_packet: {datetime.fromtimestamp(float(first_packet.time))}')
print(f'UTC converted time last_packet: {datetime.fromtimestamp(float(last_packet.time))}')
#delta_time_elapsed =
#print(f'delta p0, pLAST: {float(delta):.6f} seconds')

#########

_hrx()
print(packets)

[-] Accessing PCAP at path: {./ping_dns_test.pcapng}`... typeof packets: <class 'scapy.plist.PacketList'>
DONE.
2026-02-03 15:33:16.330169
type of p0: <class 'scapy.layers.l2.Ether'>
timestamp first_packet: 1770132796.3301687
timestamp last_packet: 1770132874.0507542
UTC converted time first_packet: 2026-02-03 15:33:16.330169
UTC converted time last_packet: 2026-02-03 15:34:34.050754




ValueError: dictionary update sequence element #0 has length 1; 2 is required

In [ ]:
packets = rdpcap(path)
print(f"typeof packets: {type(packets)}")
packets.tcp

typeof packets: <class 'scapy.plist.PacketList'>


AttributeError: 'list' object has no attribute 'tcp'

In [ ]:
from scapy.all import Packet, PacketList
from pandas import DataFrame

def count(pcap: PacketList) -> int:
  return len(pcap)

def utc_stamp(p: Packet):
  return float(p.time) # Return UTC time

### Packet Extraction

# Assumes standard {'ips':['0.0.0.0'], 'fields':['src', etc]}
def get_ips(pcap: PacketList, fields: list = None, ips: list = None) -> dict:
  if fields and ips:
    print(f'fields: {fields}')
    print(f'ips: {ips}')
    print(f'FIELDS AND IPS ARE TRUE =====================================')
  else:
    fields = []
    ips = []

  ignored = []
  ip_list = []
  for pkt in pcap:
    if IP not in pkt: # Non-IP packets (VLAN, link layer, etc)
      continue

    if 'all' in fields:
      print(f'- SEARCHING FOR {pkt[IP].src} in field(all src) ... ', end='')
      if pkt[IP].src in ips:
        print(f'FOUND in src')
        ignored.append([pkt[IP].src, 'src'])
        continue
      else:
        print(f'NOT FOUND in src')
        print(f'- SEARCHING FOR {pkt[IP].dst} in field(all dst) ... ', end='')
        if pkt[IP].dst in ips:
          print(f'FOUND in dst')
          ignored.append([pkt[IP].dst, 'dst'])
          continue
        else:
          print(f'NOT FOUND in dst')
    else:
      if 'src' in fields:
        if pkt[IP].src in ips:
          continue
      elif 'dst' in fields:
        if pkt[IP].dst in ips:
          continue
      else:
        # no fields so no matches needed
        pass

    pkt_dict = {
        'src_ip': pkt[IP].src,
        'dst_ip': pkt[IP].dst
    }
    ip_list.append(pkt_dict)

  _title(string=f'found {ignored}')
  print(f'ips: {ips}')

  return ip_list

def pop_ips_from_packetlist(target: PacketList = None, ips: list = None,
                            fields: list = None) -> None:
  pass

def pop_ips(target: dict = None, ips: list = None, verbose: bool = None) -> None:
  if target is None:
    _msg(
        msg_at='pop_ip',
        msg_str='No dict target',
        msg_type='ERROR'
    )
    return 0
  elif type(target) == dict:
    ## if target is dictionary
    if ips is None:
      _msg(
          msg_at='pop_ip',
          msg_str=f'No IP target found in {ip}',
          msg_type='ERROR'
      )
      return 0
    if verbose is None:
      verbose = False
    else:
      pre_pop_len = len(target)

    popped_ips = []
    for ip in ips:
      try:
        if verbose:
          print(f'- popping {ip} ... ', end='')
        target.pop(ip)
        popped_ips.append(ip)
        if verbose:
          print(f'DONE')
      except Exception as e:
        if verbose:
          print(f'ERROR')
          _msg(
              msg_typ='EXCEPTION',
              msg_loc='pop_ip',
              msg_str=f'Could not pop ip {ip}',
              acc_msg=e
          )
          popped_ips.append([ip, 'ERR'])
        continue
    if verbose:
      post_pop_len = len(target)
      delta_pop = pre_pop_len - post_pop_len
      print(f'Popped IPs [start={pre_pop_len}, stop={post_pop_len} ({delta_pop}). IPs={len(ips)}]:')
      for ip in popped_ips:
        print(f'- {ip}')
  elif type(target) == DataFrame:
    print(f'TYPE DATAFRAME')
  else:
    print(f'UNSUPPORTED TYPE: {type(target)}')


# input pcap/ip_dict?
# MUST IGNORE FIRST, CANT IGNORE AFTER FUNCTION EXECUTION
def get_unique_packets(pcap: PacketList, target_feature: str,
                       ignore: dict = None) -> DataFrame:
  if target_feature not in ['src_ip', 'dst_ip']:
    print(f'\n[!] ERROR:\n- `get_unique_packets`:\n-- target_feature: {target_feature} NOT [\'src_ip\', \'dst_ip\']')
    return 0

  # get ips WITH ignores
  if ignore is not None:
    ips = get_ips(
        pcap=pcap,
        ips=ignore['ips'],
        fields=ignore['fields']
    )
  # get ips WITHOUT ignores
  else:
    ips = get_ips(
        pcap=pcap
    )

  df_ips = pd.DataFrame(ips) # convert to df
  unique_ips = df_ips[target_feature].value_counts().to_dict() # store unique ips and freqs in dict

  # ignore any ips
  # if ignore is not None:
  #   pop_ips(
  #       target=unique_ips,
  #       ips=ignore['ips'],
  #       verbose=True
  #   )

  df_unique_ips = pd.DataFrame.from_dict(unique_ips, orient='index', columns=['freq']) # organize by index and freq
  df_unique_ips.index.name = target_feature # name index (above ips)
  df_unique_ips = df_unique_ips.reset_index() # move `index` to its own column, numerical idx's
  df_unique_ips.index = range(1, len(df_unique_ips) + 1) # idx start 1 not 0

  return df_unique_ips


### Utilities

def _msg(msg_loc: str = None, msg_str: str = None,
         msg_sym: chr = None, msg_typ: str = None,
         acc_msg: str = None) -> str:
  acceptable_msg_types = ['WARN', 'ERROR', 'EXCEPTION']
  if msg_typ not in acceptable_msg_types:
    _msg(
        msg_loc ='_msg func',
        msg_str = f'msg_type `{msg_typ}` not in {acceptable_msg_types}. Defaulting to `WARN`',
        msg_typ = 'WARN'
    )

  if msg_sym is None:
    if msg_typ == 'ERROR':
      msg_sym = '!'
    if msg_typ == 'WARN':
      msg_sym = '*'
    if msg_typ == 'EXCEPTION':
      msg_sym = '#'

  if msg_loc is None:
    msg_loc = 'GENERAL_LOC'

  if msg_str is None:
    msg_str = 'GENERAL_MSG'
  else:
    msg_str = f'\n[{msg_sym}] {msg_typ}:\n  - `{msg_loc}`:\n  -- {msg_str}\n'
    if acc_msg is not None:
      msg_str += f' -- {acc_msg}\n'

  print(msg_str)


def _hr(char: str = None, size: int = None) -> str:
  if char is None:
    char = '-'
  if size is None:
    size = 30
  print(f'{char*size}')

def _hrx() -> str:
  print()
  _hr(
      char = '=',
      size = 80
  )
  print()

def _title(string: str = None, char: str = None, width: int = None,
           spaces: int = None, return_val: bool = None) -> str:
  if char is None:
    char = '~'
  if width is None:
    width = 80
  if spaces is None:
    spaces = 5
  if string is None:
    string = 'default'
  if return_val is None:
    return_val = False

  formatting = [
      str.strip,
      #str.title
    ]
  for func in formatting:
    #print(f'PRE  {func.__name__}: `{string}`')
    string = func(string)
    #print(f'POST {func.__name__}: `{string}`')
    #_hr()

  string = f'{' '*spaces}{string}{' '*spaces}'
  string = f'{string:{char}^{width}}'

  if return_val:
    return string
  else:
    print(string)

## add ignore

#### testing area ####
# s = 'Breeze AI Project   '
# print(_title(s))

# #s = ' xI really LIKE RachAEl  ! '
# #_title(string=s)

# pc = packets
# u = get_unique_packets(
#     pcap=pc,
#     target_feature='src_ip'
# )
# _msg(
#     msg_at='_hr',
#     msg_typ='ERROR',
#     msg_str='yern'
# )

# pcap = packets
# ips = get_ips(pcap) # get ips
# df_ips = pd.DataFrame(ips) # convert to df
# unique_ips = df_ips['src_ip'].value_counts().to_dict() # store unique ips and freqs in dict

#   # # ignore any ips
#   # if ignore is not None:
#   #   for ip in ignore:
#   #     ## confirm check
#   #     pop_ip(unique_ips)

# print(f'KEYS: {unique_ips.keys()}')
# ip = '66.79.0.0'
# ip2 = '162.159.130.234'
# print(f'{ip2} in dict {(ip2 in unique_ips.keys())} len {len(unique_ips)}')
# pop_ip(
#     target=unique_ips,
#     ips=[ip2, ip, '192.178.218.106'],
#     verbose=True
# )
# print(f'KEYS: {unique_ips.keys()}')
# print(f'{ip2} in dict {(ip2 in unique_ips.keys())} len {len(unique_ips)}')

  # print(f'- {'192.178.218.106' in unique_ips.keys()}')
  # unique_ips.pop('192.178.218.106')
  # print(f'- {'192.178.218.106' in unique_ips.keys()}')

  # ignore ips
  # if ignore:
  #   unique_ips.pop()

  # df_unique_ips = pd.DataFrame.from_dict(unique_ips, orient='index', columns=['freq']) # organize by index and freq
  # df_unique_ips.index.name = target_feature # name index (above ips)
  # df_unique_ips = df_unique_ips.reset_index() # move `index` to its own column, numerical idx's
  # df_unique_ips.index = range(1, len(df_unique_ips) + 1) # idx start 1 not 0

pcap1 = packets
src_df = get_unique_packets(
    pcap=pcap1,
    target_feature='src_ip',
    ignore={
        'ips': [
            '142.251.179.113',
        ],
        'fields': [
            'all',
        ]
    }
)
src_df = get_unique_packets(pcap1, 'src_ip')
print(type(src_df))
#src_df = src_df.to_dict()
#print(type(src_df))
_hrx()
pop_ips(
    target=src_df,
    ips=['66.71.101.209'],
    verbose=True
)
print(src_df)

_title(string='TESTING CAP IGNORE')
# can pcap be turned into dict from rdpcap?
ips = rdpcap(packets)

fields: ['all']
ips: ['142.251.179.113']
FIELDS AND IPS ARE TRUE =====================================
- SEARCHING FOR 20.69.136.49 in field(all src) ... NOT FOUND in src
- SEARCHING FOR 66.71.101.209 in field(all dst) ... NOT FOUND in dst
- SEARCHING FOR 66.71.101.209 in field(all src) ... NOT FOUND in src
- SEARCHING FOR 20.69.136.49 in field(all dst) ... NOT FOUND in dst
- SEARCHING FOR 162.159.130.234 in field(all src) ... NOT FOUND in src
- SEARCHING FOR 66.71.101.209 in field(all dst) ... NOT FOUND in dst
- SEARCHING FOR 162.159.130.234 in field(all src) ... NOT FOUND in src
- SEARCHING FOR 66.71.101.209 in field(all dst) ... NOT FOUND in dst
- SEARCHING FOR 66.71.101.209 in field(all src) ... NOT FOUND in src
- SEARCHING FOR 162.159.130.234 in field(all dst) ... NOT FOUND in dst
- SEARCHING FOR 162.159.130.234 in field(all src) ... NOT FOUND in src
- SEARCHING FOR 66.71.101.209 in field(all dst) ... NOT FOUND in dst
- SEARCHING FOR 66.71.101.209 in field(all src) ... NOT FOUND i

AttributeError: 'list' object has no attribute 'read'

# Try to pull SRC, DST, ULT (ultimate, total) IPS

# test unique ip function, WORKING

``` python
pcap1 = packets
src_df = get_unique_packets(pcap1, 'src_ip')
dst_df = get_unique_packets(pcap1, 'dst_ip')

print(src_df)
hr()
print(dst_df)
```

# Try to pull gen uniq, WORKING

> Knowing this is important identify unexpectedly quite hosts being chattier than expected

``` python
pcap3 = packets
ips = get_ips(pcap3)
src_uniq = get_unique_packets(pcap3, target_feature='src_ip')
dst_uniq = get_unique_packets(pcap3, target_feature='dst_ip')

# to combine, we need to have identical comlumn names:
src_uniq_ren = src_uniq.rename(columns={'src_ip': 'ip', 'freq': 'freq'})
dst_uniq_ren = dst_uniq.rename(columns={'dst_ip': 'ip', 'freq': 'freq'})

gen_uniq_df = pd.concat([src_uniq_ren, dst_uniq_ren], ignore_index=True) # Aligns them linearly -- if src len = 10 and dst len = 14, col1 len now=24
gen_uniq_df = gen_uniq_df.groupby('ip', as_index=False)['freq'].sum()
gen_uniq_df = gen_uniq_df.rename(columns={'freq': 'gen_ip_freq'})

# sort by ascending for loudest endpoint
gen_uniq_df = gen_uniq_df.sort_values(by='gen_ip_freq', ascending=False).reset_index(drop=True)

gen_uniq_df
```

In [ ]:
# Try to pull specific unique and freqs of conversations
pcap2 = packets



In [ ]:
# Try to combine src and dst dfs for (`src_ip`, `src_ip_freq`, `dst_ip`, `dst_ip_freq`)
how would i do something like union to achieve this?

src_ip_df = (`src_ip`, `freq`) # data inside
dst_ip_df = (`dst_ip`, `freq`) # data inside
all_ip_df = (`src_ip`, `src_ip_freq`, `dst_ip`, `dst_ip_freq`) # combined data inside

In [ ]:
from scapy.all import IP, TCP, UDP, ICMP

pcap = packets
ips = []

for p in pcap:
  # Non-IP packets -- ARP, Link-layer, Ethernet control, VLAN tags
  if IP not in p:
    continue

  pkt = {
      'src_ip': p[IP].src,
      'dst_ip': p[IP].dst,
  }

  ips.append(pkt)

df_ips = pd.DataFrame(ips)
print(df_ips)

hr()

unique_src_ips = df_ips['src_ip'].value_counts().to_dict()
df_unique_src_ips = pd.DataFrame.from_dict(unique_src_ips, orient='index', columns=['freq'])
df_unique_src_ips.index.name = 'src_ip'
df_unique_src_ips = df_unique_src_ips.reset_index() # move `index` to its own column, numerical idx's
df_unique_src_ips.index = range(1, len(df_unique_src_ips) + 1) # idx start 1 not 0
print(df_unique_src_ips)
_hr(char='*')


for key in ult_ips:
  print(f'Key: {key}')
  for val in ult_ips[key]:
    print(f'- {val}')

               src_ip           dst_ip
0        20.69.136.49    66.71.101.209
1       66.71.101.209     20.69.136.49
2     162.159.130.234    66.71.101.209
3     162.159.130.234    66.71.101.209
4       66.71.101.209  162.159.130.234
...               ...              ...
1808    66.71.101.209    66.71.127.255
1809  162.159.130.234    66.71.101.209
1810  162.159.130.234    66.71.101.209
1811    66.71.101.209  162.159.130.234
1812   150.171.109.71    66.71.101.209

[1813 rows x 2 columns]
             src_ip  freq
1     66.71.101.209   869
2   162.159.130.234   174
3    142.251.179.97   118
4   192.178.218.106   109
5       172.64.41.4   109
6     216.239.36.21    66
7   172.253.139.101    56
8    172.253.63.113    34
9   142.251.179.113    26
10   54.246.206.151    18
11  142.251.163.154    17
12    172.253.63.95    17
13  142.251.167.139    16
14   104.70.250.213    14
15     3.162.112.51    13
16     146.75.37.91    12
17   172.66.168.139    12
18    34.144.254.29    11
19    104.18.

NameError: name 'ult_ips' is not defined

# Understanding packet-level feature extractions, DONE

## Getting random packet
``` python
import random
rand_packet_num = random.randrange(0, len(packets))
p = packets[rand_packet_num]  
```

## Show all possible fields for a layer type
``` python
from scapy.all import *
from scapy.layers.inet import IP, TCP, UDP, ICMP
features = [IP, TCP, UDP, ICMP]
for f in features:
  _title(
      string=f.__name__,
      width=40
  )
  ls(f)
_hrx()
```

## Display all layers, fields, and features of a given packet
``` python
p.show()
_hrx()
```

## Since packets are LAYERED, need to see if the layers exist in a packet first
### See what layers exist
``` python
print(f'Layers: {p.layers()}')
_hrx()
```

### From here, we can choose from each valid layer:
``` python
layers = p.layers()
for layer in layers:
  present = False
  if layer in p:
    present = True
  print(f'{layer} present: {present}')
```

---

In [ ]:
from scapy.layers.inet import IP

def packet_allowed(pkt, ignore_ips: list = None, fields: list = None) -> bool:
  # No IP associated with packet
  if IP not in pkt:
    return False

  # No ignore list
  if not ignore_ips:
    return True

  if not fields or 'all' in fields:


# get packets
packets = rdpcap(path)
packets[0]



<Ether  dst=9c:b6:d0:f6:dc:f9 src=00:00:5e:00:09:91 type=IPv4 |<IP  version=4 ihl=5 tos=0x0 len=40 id=5966 flags=DF frag=0 ttl=100 proto=6 chksum=0xbaf3 src=20.69.136.49 dst=66.71.101.209 |<TCP  sport=443 dport=54089 seq=3669221060 ack=1800276014 dataofs=5 reserved=0 flags=FA window=16386 chksum=0x2949 urgptr=0 |>>>

In [ ]:
from scapy.layers.inet import IP

def packet_allowed(pkt, ignore_ips=None, fields=None) -> bool:
    if IP not in pkt:
        return False

    if not ignore_ips:
        return True

    if not fields or 'all' in fields:
        return not (
            pkt[IP].src in ignore_ips or
            pkt[IP].dst in ignore_ips
        )

    if 'src' in fields and pkt[IP].src in ignore_ips:
        return False

    if 'dst' in fields and pkt[IP].dst in ignore_ips:
        return False
